In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
!pip install import_ipynb 
!conda install -c conda-forge import_ipynb

In [9]:
import sys

sys.path.append('../')

from game_runner import NegotitaionGame
from eval.game_evaluator import GameEvaluator
import agents.simple_agent as simple_agent
import agents.llm_agent as llm_agent
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from dataclasses import dataclass, field
from math import prod, sqrt
sys.path.append('../caif_negotiation/')

# Import the notebook
#import import_ipynb
#%run '../test_game_eval.ipynb'  
import torch
from utils.offer import Offer

from prompts.make_prompt import make_prompt
from prompts.make_prompt_bargain import make_prompt_bargain
from metrics.visualizations import (
    plot_discounted_values,
    plot_offer_evolution,
    plot_negotiation_gap,
    plot_fairness
)
import concurrent.futures


pathology_results = pd.DataFrame()  
import itertools
envy_results_history = {}
from eval.metrics import *
from utils.helpers import *
from utils.negotiation_game import *



In [10]:
import time
import pandas as pd
import torch
import numpy as np
from math import sqrt, prod

# ------------------------------------------------------------------------
# Configuration
# ------------------------------------------------------------------------
# prompt_style = 'llama_3.3_70b_maximize_value_outside_offer_cot_1_test'
prompt_style = 'openai_o1_test_new_data_object_parallel'
llm_type = 'openai'
date = '1_28_2025'
max_rounds = 3
games = 10
circles = [5, 6]



## Run Games

In [ ]:
'''
for circle in circles:
    print(f"Running game for circle {circle}")
    run_game(circle, games, max_rounds, date, prompt_style, llm_type)
'''

with concurrent.futures.ProcessPoolExecutor() as executor:
    future_to_circle = {
        executor.submit(run_game, circle, games, max_rounds, date, prompt_style, llm_type): circle
        for circle in circles
    }
    for future in concurrent.futures.as_completed(future_to_circle):
        circle_val = future_to_circle[future]
        try:
            future.result()
            print(f"[INFO] circle={circle_val} run finished successfully.")
        except Exception as exc:
            print(f"[ERROR] circle={circle_val} generated an exception: {exc}")